In [ ]:
import os
import math
import torch
from torch.autograd import Variable
from torch.optim import Adam
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, random_split, Dataset
from scipy.io import wavfile
import scipy.signal
import numpy as np

In [ ]:
NUM_CLASSES = 3
BATCH_SIZE = 1
SONG_LENGTH_SECONDS = 2

In [ ]:
class MusicDataset(Dataset):
    def __init__(self, directory, genres, add_dimension, downsample=None, noise=False):
        self.directory = directory
        self.files = []
        self.downsample = downsample
        self.add_dimension = add_dimension
        self.noise = noise
        for label, genre in enumerate(genres):
            genre_path = os.path.join(directory, genre)
            self.files.extend([(os.path.join(genre_path, f), label) for f in os.listdir(genre_path)])

    def __getitem__(self, index):
        song, label = self.files[index]
        rate, data = wavfile.read(f'{self.directory}/{song}')
        
        data = data[:44100*SONG_LENGTH_SECONDS]
        
        if self.downsample:
            data = scipy.signal.resample(data, self.downsample * SONG_LENGTH_SECONDS)

        if self.noise:
            gauss = np.random.normal(0.01, 0.001, (len(data),))
            data = data + gauss
        
        tensor = torch.Tensor(data) / (2**15)
        # add an input dimension to the data [441000] => [1, 441000]. Conv1d expects data in this format.
        if self.add_dimension:
            tensor.unsqueeze_(0)
        return tensor, torch.tensor(label, dtype=torch.long)
    
    def input_size(self):
        if self.add_dimension:
            return len(self[0][0][0])
        else:
            return len(self[0][0])
    
    def __len__(self):
        return len(self.files)



def load_dataset(add_dimension, downsample=None, noise=False):
    d = MusicDataset('.', ['rock', 'electro', 'classic'], add_dimension, downsample=downsample, noise=noise)
    train, validate = random_split(d, [900, 300])

    loader = DataLoader(train, batch_size=BATCH_SIZE)
    validation_loader = DataLoader(validate, batch_size=BATCH_SIZE)
    return d.input_size(), loader, validation_loader

In [ ]:
class Model1Linear(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.h1 = nn.Linear(input_size, hidden_size)
        self.h2 = nn.Linear(hidden_size, hidden_size)
        self.h3 = nn.Linear(hidden_size, hidden_size)
        self.h4 = nn.Linear(hidden_size, hidden_size)
        self.h5 = nn.Linear(hidden_size, hidden_size)
        self.h6 = nn.Linear(hidden_size, hidden_size)
        self.h7 = nn.Linear(hidden_size, hidden_size)
        self.h8 = nn.Linear(hidden_size, hidden_size)
        self.h9 = nn.Linear(hidden_size, NUM_CLASSES)
    
    def forward(self, x):
        x = x.data.view(-1, input_size)
         
        x = self.h1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h2(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h3(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h4(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h5(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h6(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h7(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h8(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h9(x)
        x = F.softmax(x, dim=1)
        return x

In [ ]:
class ModelConv1(nn.Module):
    def __init__(self, input_size, kernel_size=5, conv_out_channels=5, linear_size=50):
        super().__init__()

        if kernel_size % 2 != 1:
            raise Exception('Only odd kernel_size are supported')
        self.conv_out_channels = conv_out_channels
        self.conv1 = nn.Conv1d(1, conv_out_channels, kernel_size=kernel_size)
        conv_layer_output_size = int(input_size - (kernel_size - 1))

        self.pooled_samples = int(conv_layer_output_size / 2)
        self.h1 = nn.Linear(self.pooled_samples * conv_out_channels, linear_size)
        self.h2 = nn.Linear(linear_size, linear_size)
        self.h3 = nn.Linear(linear_size, linear_size)
        self.h4 = nn.Linear(linear_size, linear_size)
        self.h9 = nn.Linear(linear_size, NUM_CLASSES)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool1d(x, 2)

        x = x.view(-1, self.pooled_samples * self.conv_out_channels)

        x = self.h1(x)
        x = F.relu(x)
        
        x = self.h2(x)
        x = F.relu(x)
        
        x = self.h3(x)
        x = F.relu(x)
        
        x = self.h4(x)
        x = F.relu(x)

        x = self.h9(x)
        x = F.softmax(x, dim=1)
        return x

In [ ]:
class ModelConv2(nn.Module):
    def __init__(self, input_size, kernel_size=5, conv_out_channels=5, linear_size=50):
        super().__init__()

        if kernel_size % 2 != 1:
            raise Exception('Only odd kernel_size are supported')
        self.conv_out_channels = conv_out_channels
        self.conv1 = nn.Conv1d(1, conv_out_channels, kernel_size=kernel_size)
        self.conv2 = nn.Conv1d(conv_out_channels, conv_out_channels, kernel_size=kernel_size)
        self.conv3 = nn.Conv1d(conv_out_channels, conv_out_channels, kernel_size=kernel_size)

        #conv_layer_output_size = int(input_size - (kernel_size - 1))
        x = input_size
        x = x - (kernel_size - 1)
        x = int(x / 5)
        
        x = x - (kernel_size - 1)
        x = int(x / 5)
        
        x = x - (kernel_size - 1)
        x = int(x / 5)
        self.pooled_samples = x * conv_out_channels

        self.h1 = nn.Linear(self.pooled_samples, linear_size)
        self.h2 = nn.Linear(linear_size, linear_size)
        self.h3 = nn.Linear(linear_size, linear_size)
        self.h4 = nn.Linear(linear_size, linear_size)
        self.h9 = nn.Linear(linear_size, NUM_CLASSES)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool1d(x, 5)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool1d(x, 5)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool1d(x, 5)

        x = x.view(BATCH_SIZE, self.pooled_samples)
        x = self.h1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h2(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h3(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h4(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h9(x)
        x = F.softmax(x, dim=1)
        return x

In [ ]:
from datetime import datetime

def evalulate(model, validation_loader):
    model.eval()
    loss = 0.0
    for data, labels in validation_loader:
        predictions_per_class = model(data.cuda())
        _, highest_prediction_class = predictions_per_class.max(1)
        loss += F.nll_loss(predictions_per_class, labels.cuda())
    return loss/len(validation_loader)

def learn(model, loader, validation_loader, epochs=30, learning_rate=0.001):
    torch.cuda.empty_cache()
    optimizer = Adam(params=model.parameters(), lr=learning_rate)

    f = open(f'{datetime.now().isoformat()}.txt', 'w', buffering=1)

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        for data, labels in loader:
            predictions_per_class = model(data.cuda())
            highest_prediction, highest_prediction_class = predictions_per_class.max(1)

            # how good are we? compare output with the target classes
            loss = F.nll_loss(predictions_per_class, labels.cuda())
            total_loss += loss.item()

            model.zero_grad()
            loss.backward()
            optimizer.step()
        
        train_loss = total_loss/len(loader)
        validation_loss = evalulate(model, validation_loader)
        stats = f'Epoch: {epoch}, Train Loss: {train_loss}, Validation Loss: {validation_loss.item()}'
        print(stats)
        f.write(f'{stats}\n')
        
    return model

In [ ]:
#input_size, loader, validation_loader = load_dataset(add_dimension=False, noise=False)
#model = Model1Linear(input_size, 500).cuda()
#learn(model, loader, validation_loader, 10000, learning_rate=0.0001)

In [ ]:
input_size, loader, validation_loader = load_dataset(add_dimension=True, downsample=22050, noise=True)
model = ModelConv2(input_size, kernel_size=5, conv_out_channels=10, linear_size=500).cuda()
learn(model, loader, validation_loader, epochs=10000, learning_rate=0.0001)